# Динамика продаж по неделям из одной точки роста

Интерактивный анализ выручки магазинов по неделям работы.
Все магазины начинают с **Недели 1** (момент открытия), что позволяет сравнивать их на одинаковых этапах развития.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
from scipy import stats

## 1. Загрузка и подготовка данных

Формат входных данных:
- Первый столбец: **Магазин** (название)
- Остальные столбцы: **Нед 1, Нед 2, Нед 3, ...** (суммы продаж)

In [ ]:
# ============================================================
# ВАРИАНТ 1: Загрузка из Excel файла
# ============================================================
# df = pd.read_excel('weekly_sales.xlsx')

# ============================================================
# ВАРИАНТ 2: Демо-данные для тестирования
# ============================================================
np.random.seed(42)

stores = ['Магазин 1', 'Магазин 2', 'Магазин 3', 'Магазин 4', 'Магазин 5',
          'Магазин 6', 'Магазин 7', 'Магазин 8']
max_weeks = 24

data = {'Магазин': stores}

for store_idx, store in enumerate(stores):
    # Разная длина работы магазинов (8-24 недели)
    weeks_active = np.random.randint(8, max_weeks + 1)
    
    # Базовая выручка + тренд роста + шум
    base = np.random.uniform(80000, 150000)
    growth_rate = np.random.uniform(0.02, 0.08)  # 2-8% рост в неделю
    
    for week in range(1, max_weeks + 1):
        col_name = f'Нед {week}'
        if col_name not in data:
            data[col_name] = []
        
        if week <= weeks_active:
            # Выручка с трендом роста и случайным шумом
            value = base * (1 + growth_rate) ** (week - 1)
            noise = np.random.uniform(0.85, 1.15)
            data[col_name].append(round(value * noise))
        else:
            data[col_name].append(np.nan)  # Магазин ещё не работал

df = pd.DataFrame(data)
print(f"Загружено магазинов: {len(df)}")
print(f"Максимум недель: {max_weeks}")
df.head()

## 2. Класс для анализа недельных продаж

In [ ]:
class WeeklySalesAnalyzer:
    """
    Анализатор недельных продаж магазинов.
    Рассчитывает производные показатели и строит интерактивные графики.
    """
    
    def __init__(self, df: pd.DataFrame, store_col: str = 'Магазин'):
        """
        Args:
            df: DataFrame с колонками [Магазин, Нед 1, Нед 2, ...]
            store_col: Название колонки с магазинами
        """
        self.df = df.copy()
        self.store_col = store_col
        self.stores = df[store_col].tolist()
        
        # Определяем колонки с неделями
        self.week_cols = [c for c in df.columns if c != store_col]
        self.n_weeks = len(self.week_cols)
        
        # Матрица выручки: строки = магазины, столбцы = недели
        self.revenue_matrix = df[self.week_cols].values
        
        # Рассчитываем все производные показатели
        self._calculate_derivatives()
        
    def _calculate_derivatives(self):
        """Рассчитывает все производные показатели"""
        n_stores, n_weeks = self.revenue_matrix.shape
        
        # 1. Рост к предыдущей неделе (%)
        self.growth_pct = np.full((n_stores, n_weeks), np.nan)
        for i in range(n_stores):
            for j in range(1, n_weeks):
                prev = self.revenue_matrix[i, j-1]
                curr = self.revenue_matrix[i, j]
                if not np.isnan(prev) and not np.isnan(curr) and prev > 0:
                    self.growth_pct[i, j] = ((curr - prev) / prev) * 100
        
        # 2. Индекс роста (100% = первая неделя)
        self.growth_index = np.full((n_stores, n_weeks), np.nan)
        for i in range(n_stores):
            first_val = self.revenue_matrix[i, 0]
            if not np.isnan(first_val) and first_val > 0:
                for j in range(n_weeks):
                    curr = self.revenue_matrix[i, j]
                    if not np.isnan(curr):
                        self.growth_index[i, j] = (curr / first_val) * 100
        
        # 3. Кумулятивная сумма
        self.cumulative = np.full((n_stores, n_weeks), np.nan)
        for i in range(n_stores):
            cum_sum = 0
            for j in range(n_weeks):
                val = self.revenue_matrix[i, j]
                if not np.isnan(val):
                    cum_sum += val
                    self.cumulative[i, j] = cum_sum
        
        # 4. Скользящее среднее (4 недели)
        self.moving_avg = np.full((n_stores, n_weeks), np.nan)
        window = 4
        for i in range(n_stores):
            for j in range(window - 1, n_weeks):
                vals = self.revenue_matrix[i, j-window+1:j+1]
                if not np.any(np.isnan(vals)):
                    self.moving_avg[i, j] = np.mean(vals)
    
    def get_aggregate_stats(self, metric_matrix: np.ndarray) -> dict:
        """
        Рассчитывает агрегированную статистику по неделям
        
        Returns:
            dict с ключами: mean, median, min, max, std
        """
        stats = {
            'mean': [],
            'median': [],
            'min': [],
            'max': [],
            'std': [],
            'count': []  # кол-во магазинов с данными
        }
        
        for j in range(self.n_weeks):
            col_data = metric_matrix[:, j]
            valid_data = col_data[~np.isnan(col_data)]
            
            if len(valid_data) > 0:
                stats['mean'].append(np.mean(valid_data))
                stats['median'].append(np.median(valid_data))
                stats['min'].append(np.min(valid_data))
                stats['max'].append(np.max(valid_data))
                stats['std'].append(np.std(valid_data))
                stats['count'].append(len(valid_data))
            else:
                for key in stats:
                    stats[key].append(np.nan)
        
        return stats
    
    def get_trend_line(self, store_idx: int, metric_matrix: np.ndarray) -> tuple:
        """
        Рассчитывает линейный тренд для магазина
        
        Returns:
            (x_values, y_trend, slope, r_squared)
        """
        row = metric_matrix[store_idx]
        valid_mask = ~np.isnan(row)
        x = np.arange(self.n_weeks)[valid_mask]
        y = row[valid_mask]
        
        if len(x) < 2:
            return None, None, None, None
        
        slope, intercept, r_value, _, _ = stats.linregress(x, y)
        y_trend = slope * x + intercept
        
        return x, y_trend, slope, r_value ** 2
    
    def get_metric_data(self, metric: str) -> np.ndarray:
        """Возвращает матрицу данных для выбранной метрики"""
        metrics = {
            'Выручка': self.revenue_matrix,
            'Рост к пред. неделе (%)': self.growth_pct,
            'Индекс роста (%)': self.growth_index,
            'Кумулятивная сумма': self.cumulative,
            'Скользящее среднее (4 нед)': self.moving_avg
        }
        return metrics.get(metric, self.revenue_matrix)
    
    def to_wide_dataframe(self, metric: str = 'Выручка') -> pd.DataFrame:
        """Возвращает данные в wide-формате для отображения таблицы"""
        data = self.get_metric_data(metric)
        result = pd.DataFrame(data, columns=self.week_cols)
        result.insert(0, self.store_col, self.stores)
        return result

# Создаём анализатор
analyzer = WeeklySalesAnalyzer(df)
print(f"Анализатор создан: {len(analyzer.stores)} магазинов, {analyzer.n_weeks} недель")

## 3. Интерактивный график

In [ ]:
# Цветовая палитра для магазинов
COLORS = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5'
]

def create_weekly_chart(
    analyzer: WeeklySalesAnalyzer,
    selected_stores: list = None,
    metric: str = 'Выручка',
    show_mean: bool = True,
    show_median: bool = False,
    show_corridor: bool = False,
    show_trend: bool = False,
    week_range: tuple = None
) -> go.Figure:
    """
    Создаёт интерактивный график динамики продаж по неделям
    
    Args:
        analyzer: Экземпляр WeeklySalesAnalyzer
        selected_stores: Список выбранных магазинов (None = все)
        metric: Метрика для отображения
        show_mean: Показывать среднюю линию
        show_median: Показывать медиану
        show_corridor: Показывать коридор min-max
        show_trend: Показывать линии тренда
        week_range: (start, end) диапазон недель для отображения
    """
    # Получаем данные метрики
    data = analyzer.get_metric_data(metric)
    
    # Фильтруем магазины
    if selected_stores is None:
        selected_stores = analyzer.stores
    store_indices = [analyzer.stores.index(s) for s in selected_stores if s in analyzer.stores]
    
    # Определяем диапазон недель
    if week_range is None:
        week_start, week_end = 0, analyzer.n_weeks - 1
    else:
        week_start, week_end = week_range[0] - 1, week_range[1] - 1
    
    x_values = list(range(1, analyzer.n_weeks + 1))[week_start:week_end + 1]
    
    fig = go.Figure()
    
    # Коридор min-max (добавляем первым, чтобы был на заднем плане)
    if show_corridor:
        agg_stats = analyzer.get_aggregate_stats(data)
        min_vals = agg_stats['min'][week_start:week_end + 1]
        max_vals = agg_stats['max'][week_start:week_end + 1]
        
        # Заливка коридора
        fig.add_trace(go.Scatter(
            x=x_values + x_values[::-1],
            y=max_vals + min_vals[::-1],
            fill='toself',
            fillcolor='rgba(128, 128, 128, 0.15)',
            line=dict(color='rgba(128, 128, 128, 0)'),
            name='Коридор min-max',
            hoverinfo='skip',
            showlegend=True
        ))
    
    # Линии магазинов
    for i, store_idx in enumerate(store_indices):
        store_name = analyzer.stores[store_idx]
        y_values = data[store_idx, week_start:week_end + 1]
        color = COLORS[i % len(COLORS)]
        
        # Основная линия магазина
        fig.add_trace(go.Scatter(
            x=x_values,
            y=y_values,
            mode='lines+markers',
            name=store_name,
            line=dict(color=color, width=2),
            marker=dict(size=6),
            hovertemplate=(
                f'<b>{store_name}</b><br>'
                f'Неделя: %{{x}}<br>'
                f'{metric}: %{{y:,.0f}}<br>'
                '<extra></extra>'
            )
        ))
        
        # Линия тренда для магазина
        if show_trend:
            x_trend, y_trend, slope, r2 = analyzer.get_trend_line(store_idx, data)
            if x_trend is not None:
                # Фильтруем по диапазону недель
                mask = (x_trend >= week_start) & (x_trend <= week_end)
                if np.any(mask):
                    fig.add_trace(go.Scatter(
                        x=x_trend[mask] + 1,  # +1 для отображения с 1
                        y=y_trend[mask],
                        mode='lines',
                        name=f'{store_name} (тренд)',
                        line=dict(color=color, width=1, dash='dot'),
                        hovertemplate=(
                            f'<b>{store_name} - Тренд</b><br>'
                            f'Наклон: {slope:,.0f}/нед<br>'
                            f'R²: {r2:.2f}<br>'
                            '<extra></extra>'
                        ),
                        showlegend=False
                    ))
    
    # Средняя линия
    if show_mean:
        agg_stats = analyzer.get_aggregate_stats(data)
        mean_vals = agg_stats['mean'][week_start:week_end + 1]
        
        fig.add_trace(go.Scatter(
            x=x_values,
            y=mean_vals,
            mode='lines',
            name='Средняя',
            line=dict(color='#000000', width=3, dash='dash'),
            hovertemplate=(
                '<b>Средняя</b><br>'
                'Неделя: %{x}<br>'
                f'{metric}: %{{y:,.0f}}<br>'
                '<extra></extra>'
            )
        ))
    
    # Медиана
    if show_median:
        agg_stats = analyzer.get_aggregate_stats(data)
        median_vals = agg_stats['median'][week_start:week_end + 1]
        
        fig.add_trace(go.Scatter(
            x=x_values,
            y=median_vals,
            mode='lines',
            name='Медиана',
            line=dict(color='#E91E63', width=2, dash='dot'),
            hovertemplate=(
                '<b>Медиана</b><br>'
                'Неделя: %{x}<br>'
                f'{metric}: %{{y:,.0f}}<br>'
                '<extra></extra>'
            )
        ))
    
    # Настройки layout
    y_title = metric
    if '%' in metric:
        y_title += ' (%)'
    
    fig.update_layout(
        title=dict(
            text=f'Динамика продаж по неделям: {metric}',
            font=dict(size=18)
        ),
        xaxis=dict(
            title='Неделя работы',
            dtick=1 if len(x_values) <= 24 else 2,
            gridcolor='#e9ecef',
            zeroline=True
        ),
        yaxis=dict(
            title=y_title,
            gridcolor='#e9ecef',
            tickformat=',.0f'
        ),
        plot_bgcolor='#f8f9fa',
        paper_bgcolor='white',
        hovermode='x unified',
        legend=dict(
            x=1.02,
            y=1,
            xanchor='left',
            font=dict(size=11)
        ),
        margin=dict(l=80, r=200, t=80, b=60),
        height=550
    )
    
    # Кнопки для zoom/pan
    fig.update_layout(
        updatemenus=[
            dict(
                type='buttons',
                showactive=False,
                x=0,
                y=1.15,
                xanchor='left',
                buttons=[
                    dict(label='Сбросить zoom',
                         method='relayout',
                         args=[{'xaxis.autorange': True, 'yaxis.autorange': True}])
                ]
            )
        ]
    )
    
    return fig

# Тестовый вызов
fig = create_weekly_chart(
    analyzer,
    selected_stores=['Магазин 1', 'Магазин 2', 'Магазин 3'],
    metric='Выручка',
    show_mean=True,
    show_corridor=True
)
fig.show()

## 4. Полностью интерактивный дашборд с виджетами

In [ ]:
# Виджеты управления
metric_dropdown = widgets.Dropdown(
    options=[
        'Выручка',
        'Рост к пред. неделе (%)',
        'Индекс роста (%)',
        'Кумулятивная сумма',
        'Скользящее среднее (4 нед)'
    ],
    value='Выручка',
    description='Метрика:',
    style={'description_width': '80px'}
)

stores_select = widgets.SelectMultiple(
    options=analyzer.stores,
    value=analyzer.stores[:5],  # По умолчанию первые 5
    description='Магазины:',
    style={'description_width': '80px'},
    layout=widgets.Layout(height='150px')
)

week_slider = widgets.IntRangeSlider(
    value=[1, analyzer.n_weeks],
    min=1,
    max=analyzer.n_weeks,
    step=1,
    description='Недели:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

show_mean_cb = widgets.Checkbox(value=True, description='Средняя')
show_median_cb = widgets.Checkbox(value=False, description='Медиана')
show_corridor_cb = widgets.Checkbox(value=True, description='Коридор min-max')
show_trend_cb = widgets.Checkbox(value=False, description='Линии тренда')

# Кнопки быстрого выбора магазинов
select_all_btn = widgets.Button(description='Выбрать все', button_style='info')
clear_btn = widgets.Button(description='Очистить', button_style='warning')

def select_all(b):
    stores_select.value = tuple(analyzer.stores)
    
def clear_selection(b):
    stores_select.value = tuple()

select_all_btn.on_click(select_all)
clear_btn.on_click(clear_selection)

# Вывод графика
output = widgets.Output()

def update_chart(*args):
    with output:
        output.clear_output(wait=True)
        
        if len(stores_select.value) == 0:
            print("Выберите хотя бы один магазин")
            return
        
        fig = create_weekly_chart(
            analyzer,
            selected_stores=list(stores_select.value),
            metric=metric_dropdown.value,
            show_mean=show_mean_cb.value,
            show_median=show_median_cb.value,
            show_corridor=show_corridor_cb.value,
            show_trend=show_trend_cb.value,
            week_range=tuple(week_slider.value)
        )
        fig.show()

# Привязка обновления
metric_dropdown.observe(update_chart, names='value')
stores_select.observe(update_chart, names='value')
week_slider.observe(update_chart, names='value')
show_mean_cb.observe(update_chart, names='value')
show_median_cb.observe(update_chart, names='value')
show_corridor_cb.observe(update_chart, names='value')
show_trend_cb.observe(update_chart, names='value')

# Компоновка интерфейса
controls_row1 = widgets.HBox([metric_dropdown, week_slider])
controls_row2 = widgets.HBox([show_mean_cb, show_median_cb, show_corridor_cb, show_trend_cb])
store_controls = widgets.VBox([
    widgets.HBox([select_all_btn, clear_btn]),
    stores_select
])

dashboard = widgets.VBox([
    widgets.HTML('<h3 style="margin: 10px 0;">Настройки графика</h3>'),
    controls_row1,
    controls_row2,
    store_controls,
    output
])

display(dashboard)
update_chart()  # Начальная отрисовка

## 5. Таблица данных

In [ ]:
# Таблица выручки в wide-формате
df_wide = analyzer.to_wide_dataframe('Выручка')

# Стилизация таблицы
def style_table(df):
    """Применяет цветовую шкалу к числовым колонкам"""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    return df.style.format(
        {col: '{:,.0f}' for col in numeric_cols}
    ).background_gradient(
        cmap='RdYlGn',
        subset=numeric_cols,
        axis=None
    ).set_properties(**{
        'font-size': '11px',
        'text-align': 'right'
    }).set_properties(
        subset=[analyzer.store_col],
        **{'text-align': 'left', 'font-weight': 'bold'}
    )

print("Выручка по неделям:")
style_table(df_wide)

## 6. Сводная статистика

In [ ]:
# Рассчитываем сводку по магазинам
summary_data = []

for i, store in enumerate(analyzer.stores):
    revenue = analyzer.revenue_matrix[i]
    valid_revenue = revenue[~np.isnan(revenue)]
    
    if len(valid_revenue) == 0:
        continue
    
    # Тренд
    x_trend, y_trend, slope, r2 = analyzer.get_trend_line(i, analyzer.revenue_matrix)
    
    summary_data.append({
        'Магазин': store,
        'Недель работы': len(valid_revenue),
        'Выручка (1 нед)': valid_revenue[0],
        'Выручка (посл.)': valid_revenue[-1],
        'Рост общий (%)': ((valid_revenue[-1] / valid_revenue[0]) - 1) * 100 if valid_revenue[0] > 0 else 0,
        'Средняя выручка': np.mean(valid_revenue),
        'Всего выручка': np.sum(valid_revenue),
        'Тренд (руб/нед)': slope if slope else 0,
        'R²': r2 if r2 else 0
    })

df_summary = pd.DataFrame(summary_data)

# Стилизация сводной таблицы
df_summary.style.format({
    'Выручка (1 нед)': '{:,.0f}',
    'Выручка (посл.)': '{:,.0f}',
    'Рост общий (%)': '{:+.1f}%',
    'Средняя выручка': '{:,.0f}',
    'Всего выручка': '{:,.0f}',
    'Тренд (руб/нед)': '{:+,.0f}',
    'R²': '{:.2f}'
}).background_gradient(
    cmap='RdYlGn',
    subset=['Рост общий (%)', 'Тренд (руб/нед)'],
    axis=0
)

## 7. Экспорт данных

In [ ]:
# Экспорт в Excel с несколькими листами
def export_to_excel(analyzer, filename='weekly_sales_analysis.xlsx'):
    """Экспортирует все метрики в Excel файл"""
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        # Выручка
        analyzer.to_wide_dataframe('Выручка').to_excel(
            writer, sheet_name='Выручка', index=False
        )
        
        # Индекс роста
        analyzer.to_wide_dataframe('Индекс роста (%)').to_excel(
            writer, sheet_name='Индекс роста', index=False
        )
        
        # Кумулятивная сумма
        analyzer.to_wide_dataframe('Кумулятивная сумма').to_excel(
            writer, sheet_name='Кумулятивная сумма', index=False
        )
        
        # Сводка
        df_summary.to_excel(writer, sheet_name='Сводка', index=False)
    
    print(f"Данные экспортированы в {filename}")

# Раскомментируйте для экспорта:
# export_to_excel(analyzer)